In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

In [80]:
import requests

# Download the file manually
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
response = requests.get(url)

# Write the content to a local file
with open('titanic.csv', 'wb') as f:
    f.write(response.content)

# Load the dataset from the local file
df = pd.read_csv('titanic.csv')

print("Original Dataset:")
df.head()

Original Dataset:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [81]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [82]:
# Impute missing values
# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Numerical Imputation with the mean
num_imputer = SimpleImputer(strategy='mean')
df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

# Categorical Imputation with the most frequent value (mode)
cat_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])

# Check if missing values are all handled after imputation
print("Missing values after imputation:")
print(df.isnull().sum())

Missing values after imputation:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [83]:
categorical_cols

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [84]:
# Encoding Categorical Variables 
label_encoder = LabelEncoder()

# Iterate over each categorical column and apply LabelEncoder (except 'Survived')
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])


# Feature Scaling (Standardization) for numerical columns excluding 'Survived'
scaler = StandardScaler()

# Apply scaling only to numerical columns that are not the target ('Survived')
numerical_cols_for_scaling = [col for col in numerical_cols if col != 'Survived']  # Exclude 'Survived'
df[numerical_cols_for_scaling] = scaler.fit_transform(df[numerical_cols_for_scaling])

In [85]:
# Splitting the dataset into training and testing sets
X = df.drop('Survived', axis=1)  # Features (excluding 'Survived' column)
y = df['Survived']  # Target (Survived column)

# Split into training and testing data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Print the shapes of the training and test datasets
print("Training and Testing Data Shapes:")
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

Training and Testing Data Shapes:
X_train shape: (712, 11), X_test shape: (179, 11)
y_train shape: (712,), y_test shape: (179,)


In [86]:
# Train a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  # Initialize the model
rf_classifier.fit(X_train, y_train)  # Train the model

RandomForestClassifier(random_state=0)

In [87]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Classifier Accuracy: {accuracy:.4f}")

Random Forest Classifier Accuracy: 0.8603
